In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Dense,
    Dropout,
    BatchNormalization,
    Input,
    LeakyReLU,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
import numpy as np
import os

os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理：归一化
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 28, 28, 1)  # 调整形状以匹配模型输入
x_test = x_test.reshape(-1, 28, 28, 1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    shear_range=0.1,
    fill_mode="nearest",
)
datagen.fit(x_train.reshape(-1, 28, 28, 1))  # 数据增强需要原始形状

# 构建多层感知机（MLP）神经网络模型
model = Sequential(
    [
        Input(shape=(28, 28, 1)),  # 输入层
        
        tf.keras.layers.Flatten(),  # 展平图像
        Dense(2048),  # 隐藏层 1
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(1024),  # 隐藏层 2
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(512),  # 隐藏层 3
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(256),  # 隐藏层 4
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(128),  # 隐藏层 5
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(64),  # 隐藏层 6
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(32),  # 隐藏层 7
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),

        Dense(10, activation="softmax"),  # 输出层
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),  # 使用RMSprop优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=8, min_lr=1e-6, verbose=1
)

early_stopping = EarlyStopping(
    monitor="val_loss", patience=8, restore_best_weights=True
)

# 训练模型
history = model.fit(
    datagen.flow(x_train.reshape(-1, 28, 28, 1), y_train, batch_size=128),  # 调整批量大小并恢复原始形状
    epochs=30,  # 增加训练轮数
    validation_data=(x_test.reshape(-1, 28, 28, 1), y_test),  # 恢复原始形状
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)  # 使用调整后的测试数据形状
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_mlp_v1.keras")

2025-01-11 16:17:04.283718: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-01-11 16:17:04.283746: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-11 16:17:04.283753: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-11 16:17:04.283790: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-11 16:17:04.283811: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/30


2025-01-11 16:17:05.372256: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - ETA: 0s - loss: 1.3660 - accuracy: 0.5603

2025-01-11 16:17:20.275898: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


469/469 [==============================] - 18s 34ms/step - loss: 1.3660 - accuracy: 0.5603 - val_loss: 0.2517 - val_accuracy: 0.9338 - lr: 0.0010
Epoch 2/30
469/469 [==============================] - 15s 33ms/step - loss: 0.5328 - accuracy: 0.8513 - val_loss: 0.1630 - val_accuracy: 0.9515 - lr: 0.0010
Epoch 3/30
469/469 [==============================] - 15s 32ms/step - loss: 0.3997 - accuracy: 0.8920 - val_loss: 0.1230 - val_accuracy: 0.9649 - lr: 0.0010
Epoch 4/30
469/469 [==============================] - 15s 32ms/step - loss: 0.3354 - accuracy: 0.9089 - val_loss: 0.1057 - val_accuracy: 0.9690 - lr: 0.0010
Epoch 5/30
469/469 [==============================] - 15s 32ms/step - loss: 0.2968 - accuracy: 0.9206 - val_loss: 0.0890 - val_accuracy: 0.9730 - lr: 0.0010
Epoch 6/30
469/469 [==============================] - 15s 32ms/step - loss: 0.2725 - accuracy: 0.9268 - val_loss: 0.0761 - val_accuracy: 0.9779 - lr: 0.0010
Epoch 7/30
469/469 [==============================] - 15s 32ms/step -